# Compare EC profit redistribution

This notebook aims at comparing the profit redistribution for an Energy Community
using the package EnergyCommunity.jl and TheoryOfGames.jl

## Configuration

### Paths

In [22]:
input_file = joinpath(@__DIR__, "../data/energy_community_model.yml")  # Input file

output_file_isolated = joinpath(@__DIR__, "../results/output_file_NC.xlsx")  # Output file - model users alone
output_plot_isolated = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_NC.png")  # Output png file of plot - model users alone

output_file_combined = joinpath(@__DIR__, "../results/output_file_EC.xlsx")  # Output file - model Energy community
output_plot_combined = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_EC.pdf")  # Output png file of plot - model energy community

output_plot_sankey_agg = joinpath(@__DIR__, "../results/Img/sankey_EC.png")  # Output plot of the sankey plot related to the aggregator case
output_plot_sankey_noagg = joinpath(@__DIR__, "../results/Img/sankey_NC.png")  # Output plot of the sankey plot related to the no aggregator case

enum_mode_file = "enum_mode_datasest.jld2"  # file used to store the enumerative results
total_results_file = "total_results_file_poolmode0_poolsearch200_poolsearch200_N12.jld2"  # file to store all major results
latex_output = "latex_output_poolmode0_poolsearch200_poolsearch200_N12.txt"

overwrite_files = true  # when true, output files are overwritten

true

### Import libraries

In [23]:
#using Revise
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using TheoryOfGames
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings

### Solver configurations

In [24]:
# General optimizer
OPTIMIZER = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0, "Threads"=>10)

# Optimizer for row-generation techniques, used in the IterMode of TheoryOfGames.jl
OPTIMIZER_ROW_GENERATION = optimizer_with_attributes(Gurobi.Optimizer,
    "OutputFlag"=>1,
    "LogToConsole"=>0,
    "MIPGap"=>0.1,
    # "MIPFocus"=>1,
    "TimeLimit"=>1000,
    "LogFile"=>"gurobi_poolmode0_poolsearch200_N12.log",
    "Threads"=>10,
    # "NoRelHeurTime"=>10,
    "PoolSolutions"=>200,
    "PoolSearchMode"=>0,
    # "Crossover"=>0,  # disable crossover
)

MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("OutputFlag") => 1, MathOptInterface.RawOptimizerAttribute("LogToConsole") => 0, MathOptInterface.RawOptimizerAttribute("MIPGap") => 0.1, MathOptInterface.RawOptimizerAttribute("TimeLimit") => 1000, MathOptInterface.RawOptimizerAttribute("LogFile") => "gurobi_poolmode0_poolsearch200_N12.log", MathOptInterface.RawOptimizerAttribute("Threads") => 10, MathOptInterface.RawOptimizerAttribute("PoolSolutions") => 200, MathOptInterface.RawOptimizerAttribute("PoolSearchMode") => 0])

### Energy Community options

In [25]:
NO_AGG_GROUP = GroupANC();  # type of aggregation when the Aggregator does not belong to the coalition.
                            # options: GroupANC() or GroupNC()
BASE_GROUP = GroupNC();     # base type of aggregation (it shall be GroupNC)

## General

In [26]:
ENV["COLUMNS"] = 1000  # to print more columns in the output

1000

## Optimize the Energy Community in the COperative configuration

In [27]:
# Read data from excel file
ECModel = ModelEC(input_file, EnergyCommunity.GroupCO(), OPTIMIZER)

# Reset the user set to use all stored users (10)
reset_user_set!(ECModel)
# set_user_set!(ECModel, ["user$id" for id=1:8])

# Build the model
build_model!(ECModel)

# Optimize the model
optimize!(ECModel)

Academic license - for non-commercial use only - expires 2023-02-04


An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]


Solved model


## Calculate the reward distribution functions by row-generation techniques (IterMode in TheoryOfGames.jl)

### Define the row-generation decomposition mode by IterMode

In [28]:
iter_mode = IterMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP, optimizer=OPTIMIZER_ROW_GENERATION)

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04


IterMode(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"], EnergyCommunity.var"#utility_callback_by_subgroup#662"{EnergyCommunity.var"#objective_callback_by_subgroup#583"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#611"{ModelEC}}, EnergyCommunity.var"#objective_callback_by_subgroup#488"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Any}}, Tuple{JuMP.Containers._AxisLookup{Dict{Any, Int64}}}}}}(EnergyCommunity.var"#objective_callback_by_subgroup#583"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#611"{ModelEC}}(An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]
, EnergyCommunity.var"#objective_callback_by_subgroup#611"{ModelEC}(An Energy Community Model
Energy Community problem for a Aggregating-Non-Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "u

Model not optimized
Solved model
Model not optimized


Define coalitions to precompute when performing the iterative procedure

In [29]:
# include all coalitions having no more than preload_max_size users
preload_combs_set =  [1, length(ECModel.user_set)-1, length(ECModel.user_set)] #[1, 2, 3]  #length(ECModel.user_set)]

preload_coalitions = collect(Iterators.flatten([combinations([EC_CODE; ECModel.user_set], k) for k = preload_combs_set]))

length(preload_coalitions)

77

### Calculate reward distribution using row-generation technique

Variance Least Core using IterMode

In [30]:
tick()
varleastcore_dist_iter, min_surplus_varleastcore_iter, history_varleastcore_iter, model_dist_varleastcore_iter = var_least_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
    # best_objective_stop_option="BestObjStop",
)
time_elapsed_varleastcore_iter=tok()
println("Variance Least Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varleastcore_iter/60)")

PHASE 1: Start first least core analysis



┌ Info:  started timer at: 2022-06-24T18:20:01.916
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only -


Iteration       Upper bound     Lower bound     Tol. [%]        Benefit distribution


1                3.22e+03       -2.36e+04        113.64         [61196.22712464025, 7954.270453939913, 46050.589517308166, 5701.171990193194, 5172.216515589273, 3222.387787027983, 3487.3146465045866, 31689.01038339059, 3222.387787027983, 3222.387787027983, 56232.08282918972]


...

6                2.29e+03        2.29e+03        0.00           [30606.061224778037, 8890.061916300881, 46986.380979669135, 6636.963452554162, 6108.007977950247, 2286.596324667049, 4423.106108865555, 57091.72715233592, 4238.535245205436, 2714.732147962539, 57167.87429155069]


PHASE 1: First least core analysis ended with value 2286.596324667049
 ----
PHASE 2: Start second phase to indentify the desired objective



Iteration       Upper bound     Lower bound     Tol. [%]        Benefit distribution


7                2.29e+03        2.29e+03        0.00           [44920.105498658384, 8890.061916300889, 44920.10549868057, 6636.963452554167, 6108.007977950241, 4158.179249388975, 4423.106108865561, 51785.14852679433, 6181.54162998948, 4206.721463998179, 44920.10549865896]
PHASE 2: Completed second phase



Variance Least Core - IterMode calculated with elapsed time [min]: 44.65359430666667


Variance Core method using IterMode

In [31]:
tick()
varcore_dist_iter, min_surplus_varcore_iter, history_varcore_iter, model_dist_varcore_iter = var_in_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
    # best_objective_stop_option="BestObjStop",
)
time_elapsed_varcore_iter=tok()
println("Variance Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varcore_iter/60)")

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only -

┌ Info:  started timer at: 2022-06-24T19:04:42.154
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


1                0.00e+00       -2.41e+02        100.00         [43051.78950254409, 10249.417479108555, 43051.78950253822, 8923.559777221159, 8134.169267800187, 6444.77557405596, 6291.200356742354, 43051.7895025412, 8406.448568081492, 6493.317788665189, 43051.789502541265]


2                0.00e+00       -9.39e-09        0.93           [43112.0178624679, 10068.732398833426, 43112.01786247398, 8923.559777221157, 8073.940908356586, 6444.775574055957, 6351.428716185725, 43112.01786247016, 8346.220208637622, 6493.317788665181, 43112.01786247196]
Variance Core - IterMode calculated with elapsed time [min]: 24.464084373333332


Store results as a DataFrame

In [32]:
# vector of the users
user_set_agg = [EC_CODE; get_user_set(ECModel)]

"Auxiliary function to order the output of reward distributions and return them as vectors"
vectorize_rewards(reward_dist, users_list=user_set_agg) = [reward_dist[u] for u in users_list]

# dataframe of reward distributions for the enumerative mode
df_reward_iter = DataFrame(
    user_set=user_set_agg,
    varcore_iter=vectorize_rewards(varcore_dist_iter),
    varleastcore_iter=vectorize_rewards(varleastcore_dist_iter),
)

# dataframe of the time requirements
dict_time_iter = Dict(
    "IterMode"=>0.0,
    "varcore_iter"=>time_elapsed_varcore_iter,
    "varleastcore_iter"=>time_elapsed_varleastcore_iter,
)

df_reward_iter

,user_set,varcore_iter,varleastcore_iter
,Any,Float64,Float64
1,EC,43112.0,44920.1
2,user4,10068.7,8890.06
3,user7,43112.0,44920.1
4,user8,8923.56,6636.96
5,user9,8073.94,6108.01
6,user2,6444.78,4158.18
7,user1,6351.43,4423.11
8,user5,43112.0,51785.1
9,user3,8346.22,6181.54


## Group all results and save them

Merge results

In [33]:
df_reward = df_reward_iter # innerjoin(df_reward_enum, df_reward_iter, on=:user_set)
df_reward

,user_set,varcore_iter,varleastcore_iter
,Any,Float64,Float64
1,EC,43112.0,44920.1
2,user4,10068.7,8890.06
3,user7,43112.0,44920.1
4,user8,8923.56,6636.96
5,user9,8073.94,6108.01
6,user2,6444.78,4158.18
7,user1,6351.43,4423.11
8,user5,43112.0,51785.1
9,user3,8346.22,6181.54


In [34]:
dict_time = dict_time_iter # merge(dict_time_enum, dict_time_iter)
dict_time

Dict{String, Float64} with 3 entries:
  "varcore_iter"      => 1467.85
  "IterMode"          => 0.0
  "varleastcore_iter" => 2679.22

Save results

In [35]:
# print to latex the reward table
set_default(fmt = "%.2f", convert_unicode = false)

sorted_user_set = [EC_CODE; ["user$u" for u in 1:length(get_user_set(ECModel))]]

df_reward_mod = copy(df_reward)
sort!(df_reward_mod, [order(:user_set, by=x->findfirst(x .== sorted_user_set))])
df_reward_mod[!, 2:end] = df_reward[!, 2:end] ./ 1000  # change € unit to k€
tex_df_reward = latexify(df_reward_mod; env=:table, latex=false)

L"\begin{tabular}{ccc}
user_set & varcore_iter & varleastcore_iter\\
EC & 43.11 & 44.92\\
user1 & 10.07 & 8.89\\
user2 & 43.11 & 44.92\\
user3 & 8.92 & 6.64\\
user4 & 8.07 & 6.11\\
user5 & 6.44 & 4.16\\
user6 & 6.35 & 4.42\\
user7 & 43.11 & 51.79\\
user8 & 8.35 & 6.18\\
user9 & 6.49 & 4.21\\
user10 & 43.11 & 44.92\\
\end{tabular}
"

In [36]:
df_time = DataFrame(dict_time)[!, names(df_reward)[2:end]]./3600
df_time[!, "title"] = [L"Time [h]"]
df_time = df_time[!, ["title"; names(df_reward)[2:end]]]

,title,varcore_iter,varleastcore_iter
,LaTeXStr…,Float64,Float64
1,$Time [h]$,0.407735,0.744227


In [37]:
df_iterations = DataFrame(
    title=L"Iterations",
    varcore_iter=history_varcore_iter[end][1],
    varleastcore_iter=history_varleastcore_iter[end][1],
)

,title,varcore_iter,varleastcore_iter
,LaTeXStr…,Int64,Int64
1,$Iterations$,2,7


In [38]:
df_computational_time = vcat(df_time, df_iterations)

# print to latex the equivalent dataframe for time
tex_df_computational_time = latexify(df_computational_time; env=:table)

L"\begin{tabular}{ccc}
$title$ & $varcore_{iter}$ & $varleastcore_{iter}$\\
$Time [h]$ & $0.41$ & $0.74$\\
$Iterations$ & $2.00$ & $7.00$\\
\end{tabular}
"

In [39]:
# save latex code
open(latex_output,"w") do io
    println(io, "Reward distribution table\n\n\n")
    print(io, tex_df_reward)
    println(io, "Computational time table\n\n\n")
    print(io, tex_df_computational_time)
end

In [40]:
if !isfile(total_results_file) || overwrite_files
    jldsave(total_results_file; df_reward, dict_time, df_computational_time)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233


In [41]:
load(total_results_file)

Dict{String, Any} with 3 entries:
  "df_reward"             => 11×3 DataFrame…
  "dict_time"             => Dict("varcore_iter"=>1467.85, "IterMode"=>0.0, "varleastcore_iter"=>2679.22)
  "df_computational_time" => 2×3 DataFrame…